In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-11-10_COVID"
pairwise_filename = [x for x in os.listdir(compare_folder) if '.snp.pairwise.tsv' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
df = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])
df['sample_1'] = df['sample_1'].astype(str)
df['sample_2'] = df['sample_2'].astype(str)

In [3]:
df.head()

,sample_1,sample_2,dist
0,20069944,20069944,0
1,20069944,20069551,16
2,20069944,20091271,2
3,20069944,20081614,12
4,20069944,20267853,9


In [4]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')
metadata['id'] = metadata['id'].astype(str)

In [5]:
metadata.head()

,id,DEPARTMENT__autocolor
0,20358933,DIGESTIVO
1,20360682,DIGESTIVO
2,20367276,DIGESTIVO
3,20365371,DIGESTIVO
4,20387497,OTORRINO


In [6]:
df1 = df.merge(metadata, left_on=['sample_1'], right_on=['id'], how='left')
df1 = df1.rename(columns={'DEPARTMENT__autocolor' : 'DEPARTMENT_1'})
df1 = df1.drop(['id'], axis=1)
df1 = df1.merge(metadata, left_on=['sample_2'], right_on=['id'], how='left')
df1 = df1.rename(columns={'DEPARTMENT__autocolor' : 'DEPARTMENT_2'})
df1 = df1.drop(['id'], axis=1)
df1 = df1.fillna('DIVERSIDAD')
df1['DIFFERENT'] = df1['DEPARTMENT_1'] == df1['DEPARTMENT_2']

In [7]:
out_file = os.path.join(compare_folder, 'pairwise_department.csv')
df1.to_csv(out_file, index=False)

In [8]:
group_file = os.path.join(compare_folder, 'group_summary_0.tsv')
gdf = pd.read_csv(group_file, sep="\t")

In [9]:
gdf.head()

,group,samples,N,mean,min,max
0,1,"['20273207', '20326820', '20279371ciclos5', '20288207', '20273792', '20348783', '20279180', '20289895', '20267856', '20303453', '20338831', '20306625', '20267854', '20267853', '20253374', '20294476', '20327059', '20272650', '20279189', '20326029', '20261163', '20258070', '20273218ciclos5', '20293183', '20294477', '20286872', '20273588', '20298003', '20277322', '20267855', '20269026', '20286419', '20272101', '20279372ciclos5', '20242010BIS', '20242010', '20273601', '20261728', '20279174', '20273789', '20258214', '20247604', '20255915', '20247878', '20348780', '20297088', '20407339', '20371405', '20297211']",49,0.0,0.0,0.0
1,2,"['20089518', '20069973', '20076540', '20069592', '20066483', '20069688', '20091993', '20077703', '20062229', '20089075', '20091198', '20131291', '20095471-5', '20095471', '20075029', '20077274', '20082081', '20066510', '20072745', '20072711', '20072993', '20085890', '20069707', '20164346-8', '20089370', '20155764', '20158605', '20089542', '20089209', '20069961', '20072734', '20089037', '20131360', '20069944', '20069560', '20072166', '20097074Micro', '20095471-8']",38,0.0,0.0,0.0
2,3,"['20091211', '20069624', '20072758', '20115546', '20089308', '20089300', '20091210', '20072747', '20089033', '20137499', '20062563', '20069703', '20089950', '20066486', '20069433', '20089345', '20091364', '20068881', '20091212', '20084535', '20115171', '20092775', '20077060', '20089945', '20069997', '20069843']",26,0.0,0.0,0.0
3,4,"['20066718', '20063068', '20063072', '20068171', '20060104', '20091425', '20089194', '20062682', '20056952', '20060908', '20057661', '20066411', '20205319', '20063795', '20066705', '20057660', '20063156']",17,0.0,0.0,0.0
4,5,"['20255928', '20325231', '20315305', '20322499', '20393904', '20298756', '20411437', '20298004']",8,0.0,0.0,0.0


In [177]:
def listDepartment(sample_list, metadata):
    if type(sample_list) is str:
        sample_list = sample_list.strip("][").split(', ')
        sample_list = [x.strip("'") for x in sample_list]
    list_department = []
    only_department = []
    metadata_samples = [str(x) for x in metadata.id.tolist()]
    for sample in sample_list:
        if str(sample) in metadata_samples:
            department = metadata[metadata.id == sample]['DEPARTMENT__autocolor'].values[0]
        else:
            department = 'DIVERSIDAD'
        list_department.append(str(sample) + ":" + department)
        only_department.append(department)
    counter_department = dict((x,only_department.count(x)) for x in set(only_department))

    return list_department, counter_department


In [178]:
gdf[['sample_department', 'only_department']] = gdf.apply(lambda x: listDepartment(x['samples'], metadata), axis=1, result_type="expand")

['20273207', '20326820', '20279371ciclos5', '20288207', '20273792', '20348783', '20279180', '20289895', '20267856', '20303453', '20338831', '20306625', '20267854', '20267853', '20253374', '20294476', '20327059', '20272650', '20279189', '20326029', '20261163', '20258070', '20273218ciclos5', '20293183', '20294477', '20286872', '20273588', '20298003', '20277322', '20267855', '20269026', '20286419', '20272101', '20279372ciclos5', '20242010BIS', '20242010', '20273601', '20261728', '20279174', '20273789', '20258214', '20247604', '20255915', '20247878', '20348780', '20297088', '20407339', '20371405', '20297211']
['20089518', '20069973', '20076540', '20069592', '20066483', '20069688', '20091993', '20077703', '20062229', '20089075', '20091198', '20131291', '20095471-5', '20095471', '20075029', '20077274', '20082081', '20066510', '20072745', '20072711', '20072993', '20085890', '20069707', '20164346-8', '20089370', '20155764', '20158605', '20089542', '20089209', '20069961', '20072734', '20089037'

In [182]:
gdf.to_csv(os.path.join(compare_folder, 'group_0_diversity.tsv'), sep="\t", index=False)

In [ ]:
counter_department = dict((x,only_department.count(x)) for x in set(only_department))